In [1]:

import sys, os
from scipy import stats
import wandb
import numpy as np
import networkx as nx
from random import shuffle
import matplotlib.pyplot as plt
from collections import OrderedDict
import json,pickle
from rdkit import RDLogger
from rdkit import Chem
from rdkit.Chem import MolFromSmiles
from lifelines.utils import concordance_index

# Model
import torch
import torch.nn as nn
from torch_geometric.data import InMemoryDataset, DataLoader, Batch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import TransformerConv,GATConv, GCNConv,global_max_pool as gmp, global_add_pool as gap,global_mean_pool as gep,global_sort_pool
from torch_geometric.utils import dropout_adj
from torch_geometric import data as DATA
import torchvision.transforms as T
import warnings


/home/dagaa/anaconda3/envs/graphdta/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
warnings.filterwarnings("ignore")
wandb.init(project="LEP-AD", entity="daga06")

dataset = 'davis'
# load dataset
dataset_path = 'data/' + dataset + '/'

ligands = json.load(open(dataset_path + 'drug_dict.txt'), object_pairs_hook=OrderedDict)
proteins = json.load(open(dataset_path + 'protein_dict.txt'), object_pairs_hook=OrderedDict)

In [ ]:
drugs = []
drug_smiles = []

RDLogger.DisableLog('rdApp.*')
# smiles
for d in ligands.keys():
    lg = Chem.MolToSmiles(Chem.MolFromSmiles(ligands[d]), isomericSmiles=True)
    drugs.append(lg)
    drug_smiles.append(ligands[d])

In [ ]:
# one ont encoding
def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        # print(x)
        raise Exception('input {0} not in allowable set{1}:'.format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))


def one_of_k_encoding_unk(x, allowable_set):
    '''Maps inputs not in the allowable set to the last element.'''
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))
    
# mol atom feature for mol graph
def atom_features(atom):
    # 44 +11 +11 +11 +1
    return np.array(one_of_k_encoding_unk(atom.GetSymbol(),
                                          ['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na', 'Ca', 'Fe', 'As',
                                           'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se',
                                           'Ti', 'Zn', 'H', 'Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr', 'Cr',
                                           'Pt', 'Hg', 'Pb', 'X']) +
                    one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    one_of_k_encoding_unk(atom.GetTotalNumHs(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    [atom.GetIsAromatic()])

In [3]:

def smile_to_graph(smile):
    mol = Chem.MolFromSmiles(smile)

    c_size = mol.GetNumAtoms()

    features = []
    for atom in mol.GetAtoms():
        feature = atom_features(atom)
        features.append(feature / sum(feature))

    edges = []
    for bond in mol.GetBonds():
        edges.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
    g = nx.Graph(edges).to_directed()
    edge_index = []
    mol_edge_weight=[]
    mol_adj = np.zeros((c_size, c_size))
    for e1, e2 in g.edges:
        mol_adj[e1, e2] = 1
        # edge_index.append([e1, e2])
    mol_adj += np.matrix(np.eye(mol_adj.shape[0]))
    index_row, index_col = np.where(mol_adj >= 0.5)
    for i, j in zip(index_row, index_col):
        edge_index.append([i, j])
        mol_edge_weight.append([1])
    # print('smile_to_graph')
    # print(np.array(features).shape)
    return c_size, features, edge_index,mol_edge_weight

In [10]:
# 14min
compound_iso_smiles = drugs
if not os.path.exists('data/'+dataset+'/temp/smile_graph.pickle'):
    os.mkdir('data/'+dataset+'/temp')
    # create smile graph
    smile_graph = {}
    i = 0
    for smile in compound_iso_smiles:
        g = smile_to_graph(smile)
        smile_graph[smile] = g
        if i%1000==0:
            print(i)
        i+=1
    with open('data/'+ dataset +'/temp/smile_graph.pickle','wb') as handle:
        pickle.dump(smile_graph, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
with open('data/'+ dataset +'/temp/smile_graph.pickle','rb') as f:
    smile_graph = pickle.load(f)

In [11]:
if not os.path.exists('data/'+dataset+'/temp/protein_rep.pickle'):
    esm_emb_path = 'data/' + dataset + '/proteins_emb_esm2/UniRef50_'
    protein_dict = json.load(open('data/davis/protein_dict.txt'))
    target_reps_dict = {}
    i=0
    for key in proteins.keys():
        target_reps_dict[protein_dict[key]] = torch.load(esm_emb_path+ key + '.pt')['mean_representations'][36]
        if i%1000==0:
            print(i)
        i+=1

    with open('data/'+ dataset +'/temp/protein_rep.pickle','wb') as handle:
        pickle.dump(target_reps_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('data/'+ dataset +'/temp/protein_rep.pickle', 'rb') as handle:
    target_reps_dict = pickle.load(handle)

In [4]:


# initialize the dataset
class DTADataset(InMemoryDataset):
    def __init__(self, root='/tmp', dataset='davis',
                 xd=None, y=None, transform= None,
                 pre_transform=None, smile_graph=None, target_key=None, target_rep=None):
        super(DTADataset, self).__init__(root, transform, pre_transform)
        self.dataset = dataset
        self.process(xd, target_key, y, smile_graph, target_rep)

    @property
    def raw_file_names(self):
        pass
        # return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return [self.dataset + '_data_mol.pt', self.dataset + '_data_pro.pt']

    def _process(self):
        if not os.path.exists(self.processed_dir):
            os.makedirs(self.processed_dir)

    def process(self, xd, target_key, y, smile_graph, target_rep):
        assert (len(xd) == len(target_key) and len(xd) == len(y)), 'The three lists must be the same length!'
        data_list_mol = []
        data_list_pro = []
        data_len = len(xd)
        for i in range(data_len):
            entity1 = xd[i]

            labels = y[i]
            if entity1 in smile_graph.keys():
                c_size, features, edge_index,edge_weight = smile_graph[entity1]
            else:
                c_size, features, edge_index,edge_weight = smile_to_graph(entity1)
            GCNData_mol = DATA.Data(x=torch.Tensor(np.array(features)),
                                    edge_index=torch.LongTensor(edge_index).transpose(1, 0),
                                    y=torch.FloatTensor([labels])
                                    )
            GCNData_mol.__setitem__('c_size', torch.LongTensor([c_size]))
            data_list_mol.append(GCNData_mol)

            data_list_pro.append(torch.Tensor(target_rep[target_key[i]]))
            if i%10000==0:
                print(i)
            
        if self.pre_filter is not None:
            data_list_mol = [data for data in data_list_mol if self.pre_filter(data)]
        if self.pre_transform is not None:
            data_list_mol = [self.pre_transform(data) for data in data_list_mol]
        self.data_mol = data_list_mol
        self.data_pro = data_list_pro

    def __len__(self):
        return len(self.data_mol)

    def __getitem__(self, idx):
        return self.data_mol[idx], self.data_pro[idx]
        
def collate(batch):
    graphs = Batch.from_data_list([item[0] for item in batch])
    tensors = [item[1] for item in batch]
    tensors = torch.stack(tensors)

    return graphs,tensors

In [ ]:
df_train_fold = pd.read_csv('data/' + dataset + '/'+ dataset+'_' + 'train' + '.csv')
train_drugs, train_prot_keys, train_Y = list(df_train_fold['compound_iso_smiles']), list(df_train_fold['target_sequence']), list(df_train_fold['affinity'])
train_drugs, train_prot_keys, train_Y = np.asarray(train_drugs), np.asarray(train_prot_keys), np.asarray(train_Y)

train_dataset = DTADataset(root='data', dataset=dataset + '_' + 'train', xd=train_drugs, target_key=train_prot_keys,
                            y=train_Y, smile_graph=smile_graph, target_rep=target_reps_dict)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

cuda_name = 'cuda:0'
print('cuda_name:', cuda_name)
fold = [0, 1, 2, 3, 4][0]
cross_validation_flag = True

TRAIN_BATCH_SIZE = 512
TEST_BATCH_SIZE = 512
LR = 0.0005
NUM_EPOCHS = 3000
from sklearn.model_selection import train_test_split
train_data,valid_data=train_test_split(train_dataset,shuffle=True,test_size=0.2)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=TRAIN_BATCH_SIZE, shuffle=True,num_workers=8,
                                            collate_fn=collate)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=TEST_BATCH_SIZE, shuffle=False,num_workers=4,
                                            collate_fn=collate)
                                            
# next(iter(train_loader))

In [ ]:
# GCN based model
class GNNNet(torch.nn.Module):
    def __init__(self, n_output=1, num_features_mol=78, output_dim=128, dropout=0.2):
        super(GNNNet, self).__init__()

        print('GNNNet Loaded')
        self.n_output = n_output
        self.mol_conv1 = TransformerConv(num_features_mol, num_features_mol,heads=4)
        self.mol_conv2 = TransformerConv(4*num_features_mol, num_features_mol * 2,heads=2)
        self.mol_conv3 = TransformerConv(num_features_mol * 4, num_features_mol * 4,heads=1)
        self.mol_fc_g1 = torch.nn.Linear(num_features_mol * 4, 1024)
        self.mol_fc_g2 = torch.nn.Linear(1024, output_dim)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

        # combined layers
        self.fc1 = nn.Linear(2688, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.out = nn.Linear(512, self.n_output)

    def forward(self, data_mol, data_pro):
        mol_x, mol_edge_index, mol_batch = data_mol.x, data_mol.edge_index, data_mol.batch

        x = self.mol_conv1(mol_x, mol_edge_index)
        x = self.relu(x)

        x = self.mol_conv2(x, mol_edge_index)
        x = self.relu(x)

        x = self.mol_conv3(x, mol_edge_index)
        x = self.relu(x)

        x = gep(x, mol_batch)  # global pooling

        # flatten
        x = self.relu(self.mol_fc_g1(x))
        x = self.dropout(x)
        x = self.mol_fc_g2(x)
        x = self.dropout(x)

        # concat
        xc = torch.cat((x, data_pro), 1)
        # add some dense layers
        xc = self.fc1(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        xc = self.fc2(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        out = self.out(xc)
        return out



In [ ]:
print('Learning rate: ', LR)
print('Epochs: ', NUM_EPOCHS)

models_dir = 'models'
results_dir = 'results'

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(results_dir):
    os.makedirs(results_dir)

# Main program: iterate over different datasets
result_str = ''
USE_CUDA = torch.cuda.is_available()
device = torch.device(cuda_name if USE_CUDA else 'cpu')
model = GNNNet()
model.to(device)

model_st = GNNNet.__name__
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
# training function at each epoch
scaler = torch.cuda.amp.GradScaler()
def train(model, device, train_loader, optimizer, epoch):
    print('Training on {} samples...'.format(len(train_loader.dataset)))
    model.train()
    LOG_INTERVAL = 100
    TRAIN_BATCH_SIZE = 512
    loss_fn = torch.nn.MSELoss()
    
    for batch_idx, data in enumerate(train_loader):
        data_mol = data[0].to(device)
        data_pro = data[1].to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(data_mol, data_pro)
            labels = data_mol.y.view(-1, 1)
            loss = loss_fn(output, labels)
        scaler.scale(loss).backward()
        wandb.log({"loss per batch": loss})
        #optimizer.step()
        scaler.step(optimizer)
        scaler.update()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,
                                                                           batch_idx * TRAIN_BATCH_SIZE,
                                                                           len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader),
                                                                           loss.item()))

# predict
def predicting(model, device, loader):
    model.eval()
    total_preds = torch.Tensor()
    total_labels = torch.Tensor()
    print('Make prediction for {} samples...'.format(len(loader.dataset)))
    with torch.no_grad():
        for data in loader:
            data_mol = data[0].to(device)
            data_pro = data[1].to(device)
            output = model(data_mol, data_pro)
            labels = data_mol.y.view(-1, 1)
            total_preds = torch.cat((total_preds, output.cpu()), 0)

            total_labels = torch.cat((total_labels, labels.cpu()), 0)
    return total_labels.numpy().flatten(), total_preds.numpy().flatten()

In [ ]:






def load_model(model_path):
    model = torch.load(model_path)
    return model


import time
def calculate_metrics(Y, P, dataset='davis'):
    # # aupr = get_aupr(Y, P)
    # t = time.time()
    
    
    # cindex = get_cindex(Y, P)
    # print(cindex)
    # print(concordance_index(Y, P))  # DeepDTAget_cindex(Y, P)
    
    
    cindex2 = concordance_index(Y, P)  # GraphDTA
    rm2 = get_rm2(Y, P)  # DeepDTA
    mse = get_mse(Y, P)
    # pearson = get_pearson(Y, P)
    # spearman = get_spearman(Y, P)
    # rmse = get_rmse(Y, P)

    print('metrics for ', dataset)
    # print('aupr:', aupr)
    # print('cindex:', cindex)
    print('cindex2', cindex2)
    print('rm2:', rm2)
    print('mse:', mse)
    result_str = ''
    result_str += dataset + '\r\n'
    result_str += ' ' + ' mse:' + str(mse) + ' ' + ' '+ ' ' + 'ci:' + str(cindex2)
    print(result_str)
    return mse,cindex2,rm2

def plot_density(Y, P, fold=0, dataset='davis'):
    plt.figure(figsize=(10, 5))
    plt.grid(linestyle='--')
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.scatter(P, Y, color='blue', s=40)
    plt.title('density of ' + dataset, fontsize=30, fontweight='bold')
    plt.xlabel('predicted', fontsize=30, fontweight='bold')
    plt.ylabel('measured', fontsize=30, fontweight='bold')
    # plt.xlim(0, 21)
    # plt.ylim(0, 21)
    if dataset == 'davis':
        plt.plot([5, 11], [5, 11], color='black')
    else:
        plt.plot([6, 16], [6, 16], color='black')
    # plt.legend()
    plt.legend(loc=0, numpoints=1)
    leg = plt.gca().get_legend()
    ltext = leg.get_texts()
    plt.setp(ltext, fontsize=12, fontweight='bold')
    plt.savefig(os.path.join('results', dataset + '_' + str(fold) + '.png'), dpi=500, bbox_inches='tight')


    # plot_density(Y, P, fold, dataset)
import numpy as np
import subprocess
from math import sqrt
from sklearn.metrics import average_precision_score
from scipy import stats


def get_aupr(Y, P, threshold=7.0):
    # print(Y.shape,P.shape)
    Y = np.where(Y >= 7.0, 1, 0)
    P = np.where(P >= 7.0, 1, 0)
    aupr = average_precision_score(Y, P)
    return aupr


def get_cindex(Y, P):
    summ = 0
    pair = 0

    for i in range(1, len(Y)):
        for j in range(0, i):
            if i is not j:
                if (Y[i] > Y[j]):
                    pair += 1
                    summ += 1 * (P[i] > P[j]) + 0.5 * (P[i] == P[j])

    if pair != 0:
        return summ / pair
    else:
        return 0


def r_squared_error(y_obs, y_pred):
    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)
    y_obs_mean = [np.mean(y_obs) for y in y_obs]
    y_pred_mean = [np.mean(y_pred) for y in y_pred]

    mult = sum((y_pred - y_pred_mean) * (y_obs - y_obs_mean))
    mult = mult * mult

    y_obs_sq = sum((y_obs - y_obs_mean) * (y_obs - y_obs_mean))
    y_pred_sq = sum((y_pred - y_pred_mean) * (y_pred - y_pred_mean))

    return mult / float(y_obs_sq * y_pred_sq)


def get_k(y_obs, y_pred):
    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)

    return sum(y_obs * y_pred) / float(sum(y_pred * y_pred))


def squared_error_zero(y_obs, y_pred):
    k = get_k(y_obs, y_pred)

    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)
    y_obs_mean = [np.mean(y_obs) for y in y_obs]
    upp = sum((y_obs - (k * y_pred)) * (y_obs - (k * y_pred)))
    down = sum((y_obs - y_obs_mean) * (y_obs - y_obs_mean))

    return 1 - (upp / float(down))


def get_rm2(ys_orig, ys_line):
    r2 = r_squared_error(ys_orig, ys_line)
    r02 = squared_error_zero(ys_orig, ys_line)

    return r2 * (1 - np.sqrt(np.absolute((r2 * r2) - (r02 * r02))))


def get_rmse(y, f):
    rmse = sqrt(((y - f) ** 2).mean(axis=0))
    return rmse


def get_mse(y, f):
    mse = ((y - f) ** 2).mean(axis=0)
    return mse


def get_pearson(y, f):
    rp = np.corrcoef(y, f)[0, 1]
    return rp


def get_spearman(y, f):
    rs = stats.spearmanr(y, f)[0]
    return rs


def get_ci(y, f):
    ind = np.argsort(y)
    y = y[ind]
    f = f[ind]
    i = len(y) - 1
    j = i - 1
    z = 0.0
    S = 0.0
    while i > 0:
        while j >= 0:
            if y[i] > y[j]:
                z = z + 1
                u = f[i] - f[j]
                if u > 0:
                    S = S + 1
                elif u == 0:
                    S = S + 0.5
            j = j - 1
        i = i - 1
        j = i - 1
    ci = S / z
    return ci

In [ ]:
#188 minutes
best_mse = 1000
best_test_mse = 1000
best_epoch = -1
# model_file_name = 'models/model_' + model_st + '_' + dataset + '_' + str(fold) + '.model'

for epoch in range(NUM_EPOCHS):
    train(model, device, train_loader, optimizer, epoch + 1)
    print('predicting for valid data')
    G, P = predicting(model, device, valid_loader)
    val_mse,val_ci,val_rm2 = calculate_metrics(G, P, dataset)
    wandb.log({"val_ci": val_ci})
    wandb.log({"val_mse": val_mse})
    wandb.log({"val_rm2": val_rm2})

    print('predicting for test data')
    
    if val_mse < best_mse:
        best_mse = val_mse
        best_epoch = epoch + 1
        # torch.save(model.state_dict(), model_file_name)
        print('rmse improved at epoch ', best_epoch, '; best_test_mse', best_mse, model_st, dataset, fold)
    else:
        print('No improvement since epoch ', best_epoch, '; best_test_mse', best_mse, model_st, dataset, fold)


#reaching optimzation
torch.save({
    'epoch':epoch,
    'model_state_dict':model.state_dict(),
    'optimizer_state_dict':optimizer.state_dict()
},'saved_models/esm-2-stitch.pt')

        

In [ ]:
df_test_fold = pd.read_csv('data/' + dataset + '/'+ dataset+'_' + 'test' + '.csv')
test_drugs, test_prot_keys, test_Y = list(df_test_fold['compound_iso_smiles']), list(df_test_fold['target_sequence']), list(df_test_fold['affinity'])
test_drugs, test_prot_keys, test_Y = np.asarray(test_drugs), np.asarray(test_prot_keys), np.asarray(test_Y)

test_data = DTADataset(root='data', dataset=dataset + '_' + 'test', xd=test_drugs, target_key=test_prot_keys,
                            y=test_Y, smile_graph=smile_graph, target_rep=target_reps_dict)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=TEST_BATCH_SIZE, shuffle=False,num_workers=4,
                                            collate_fn=collate)
G, P = predicting(model, device, test_loader)
test_mse,test_ci,test_rm2 = calculate_metrics(G, P, dataset)